In [10]:
import pandas as pd
import numpy as np
import time
import shutil
import os
import glob
import xlwings as xw
import argparse
from pathlib import Path
from datetime import date, timedelta, datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [23]:
print('xw: ' + xw.__version__)
print('pd: ' + pd.__version__)
print('np: ' + np.__version__)

xw: 0.29.1
pd: 2.0.3
np: 1.24.3


AttributeError: type object 'Path' has no attribute '__version__'

Read xls file then convert it into xlsx file

In [4]:
with open('D:\SGC\TPTS_Project\df_portal_order_check\daily_xls\DFP_FreshFood_Order_01_04_2024.xls', 'r') as f:
    for line in 
    print(f.readline())

,,,,,,,,151,155,161,505,159,160,130,301,153,154,157,133,136,135,134,141,152,175,162,180,178,186,511,509,510,196,304,506,508,524,530,305,541,306,158,556,561,549,565,570,4201,4202,399,220,218,234,240,217,251,293,255,239,404,245,246,403,272,9503,279,625,233,237,212,213,226,228,229,236,410,256,259,261,262,269,277,278,280,286,287,211,214,215,221,225,227,230,238,409,243,244,275,247,249,254,263,267,276,270,281,282,284,289,290,291,294,296,298,400,401,402,405,406,626,629,638,630,631,632,633,636,634,635,637,639,640,641,642,643,644,645,647,648,652,653,655,654,656,657,658,659,665,669,671,661,674,676,678,683,684,690,9302,685,687,691,688,689,692,693,695,696,9303,697,698,2001,2002,2006,9205,2004,2005,9206,2007,2008,2009,2010,2011,9208,2014,2015,2016,694,2017,2018,2019,2021,2033,2023,2025,2027,2028,2029,2030,2031,9210,2032,2034,2035,2039,2041,2043,2042,2040,2045,2046,2047,2048,2051,2052,2056,2057,2059,2060,9309,2063,2064,2065,9311,2066,9313,2069,2072,9314,2073,9315,2074,2075,2076,2077,2078,2079,2080,9

In [9]:
def convert_xls(input_path, output_path):
    try:
        app = xw.App(visible=False)
        wb = app.books.open(input_path)
        
        # Save the file as xlsx
        wb.save(output_path)

        # Close the workbook and quit Excel
        wb.close()
        app.quit()
        return True
    except:
        return False
    
convert_xls('D:\P.Truong\Checking_Data\TPTS_Project\df_portal_order_check\daily_xls\DFP_FreshFood_Order_01_04_2024.xls',\
            'D:\P.Truong\Checking_Data\TPTS_Project\df_portal_order_check\daily_xlsx\DFP_FreshFood_Order_01_04_2024.xlsx')

True

In [6]:
import xlwings as xw

# Read xls file then convert it into xlsx file
def convert_xls(input_path, output_path):
    try:
        try:
            app = xw.App(visible=False)
            wb = app.books.open(input_path)
        except:
            print('Cannot open file')
        # Save the file as xlsx
        wb.save(output_path)

        # Close the workbook and quit Excel
        wb.close()
        app.quit()
        return True
    except:
        return False



c:\Users\Admin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Đọc file và tiền xử lý, sau đó lưu vào df

In [3]:
# Rewrite the header
""""
This function is used to transfrom the original dataframe with 
inapproriate columns name and structure.

Input: raw df read from excel files. See excel files to explore the structure
Output: df with columns that has been renamed, and

"""
def clean_df(df):
    col_rename = df.loc[0].to_list()[:8]
    for i, col in enumerate(df.columns):
        if i == 8:
            break
        df.rename(columns={col: col_rename[i]},inplace=True)
    df = df.drop(index=0)
    df_melt = pd.melt(df, id_vars=df.columns[0:8], value_vars=df.columns[8:],
                  var_name = 'Store', value_name='Order_Qty')
    return df_melt


Số SKU mỗi store đặt

In [4]:
# df_melt.dropna(subset='Order_Qty')
def store_group(df):
    nSKU_by_Store = df.groupby(by='Store')['Order_Qty'].count().sort_values(ascending=False).reset_index()
    stores_ordered_today = nSKU_by_Store[nSKU_by_Store['Order_Qty']>0]
    stores_has_no_order = nSKU_by_Store[nSKU_by_Store['Order_Qty']==0]
    return stores_ordered_today, stores_has_no_order

Function to get order list of an input store

In [5]:
def order_list(store,df):
    return df[(df['Store'] == store) &
                   (~df['Order_Qty'].isna())]

Function of exporting a df to csv file

In [6]:
def export_df(df, filename):
    # today = datetime.now().strftime(format='%d-%m')
    df.to_excel('output_files/' + filename + '.xlsx', index=False)

In [7]:
def main(order_file_path = '', order_store = False, no_order_store = False, store_list = []):
    date_string = (date.today() + timedelta(1)).strftime(format='%d_%m_%Y')
    if order_file_path == '':
        xls_filename = 'DFP_FreshFood_Order_' + date_string + '.xls'
        # Get the path to today's downloaded xls file
        xls_file_path = os.getcwd() + '\\daily_xls\\' + xls_filename
        # Get the path to the intended output with xlsx extension:
        xlsx_file_path = Path(os.getcwd() + '\\daily_xlsx\\' + xls_filename).with_suffix(".xlsx")
    else:
        xls_file_path = order_file_path
        xlsx_file_path = str(xls_file_path).replace('xls','xlsx')

    if not convert_xls(input_path=xls_file_path, output_path=xlsx_file_path):
        print(xlsx_file_path)
        print(xls_file_path)
        print('Fail to convert xls file')
        return
    else:
        df = clean_df(pd.read_excel(xlsx_file_path))
        # date = str(xlsx_file_path)[-15:-5]
        if order_store:
            store_with_orders, store_without_orders = store_group(df)
            export_df(store_with_orders, date_string + '_store_có_đặt')
        if no_order_store:
            store_with_orders, store_without_orders = store_group(df)
            export_df(store_without_orders, date_string +'_store_không_đặt')
        if store_list.__len__() != 0:
            for store in store_list:
                order_by_store = order_list(store, df)
                export_df(order_by_store, date_string + '_' + str(store))

In [8]:

main(no_order_store=True, order_store=True)

In [9]:
# D:\SGC\TPTS_Project\df_portal_order_check\daily_xls\DFP_FreshFood_Order_05_04_2024.xls